In [11]:
from database import database
from matplotlib import pyplot as plt
import sklearn.feature_extraction.image
from sklearn.feature_extraction.image import extract_patches
import tables
import random
from SRCNN import run
from build_model import build_model
import tensorflow as tf

In [2]:
random.seed(31415926)
args = {}
args['filedir'] = 'train_data'
args['pattern'] = '*.bmp'
args['database_name'] = 'mock'
args['export_dir'] = 'export'
args['patch_shape'] = (64,64,3)
args['stride_size'] = 32



data_set = database(**args)
data_set.initialize()

In [3]:
run(data_set, num_epoch = 1, batch_size = 13)

Preparing data...
Instructions for updating:
Use `tf.global_variables_initializer` instead.
EPOCH: 0


In [6]:
print(data_set.size('train'))

2946


In [7]:
img_size,label_size=data_set.peek('train')
print(img_size,img_size)

(2946, 64, 64, 3) (2946, 64, 64, 3)


In [8]:
build_model(data_set.patch_shape[0], data_set.patch_shape[0], data_set.patch_shape[2])

{'images': <tf.Tensor 'images_1:0' shape=(?, 64, 64, 3) dtype=float32>,
 'ground_images': <tf.Tensor 'ground_images_1:0' shape=(?, 64, 64, 3) dtype=float32>,
 'cnn': <tf.Tensor 'add_5:0' shape=(?, 64, 64, 3) dtype=float32>,
 'saver': <tensorflow.python.training.saver.Saver at 0x239e4a8fcf8>}

In [10]:
type()

numpy.ndarray

In [27]:
a=tf.convert_to_tensor(data_set['train',1][0], dtype=tf.uint8)

AttributeError: 'numpy.ndarray' object has no attribute 'to'

In [28]:
sess = tf.InteractiveSession()
a.eval()

c:\users\ciel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


array([[[ 11,  47,  51],
        [ 11,  47,  51],
        [  9,  48,  52],
        ...,
        [117, 201, 189],
        [125, 205, 192],
        [125, 205, 192]],

       [[ 11,  47,  51],
        [ 11,  47,  51],
        [  9,  48,  52],
        ...,
        [117, 201, 189],
        [125, 205, 192],
        [125, 205, 192]],

       [[ 15,  40,  52],
        [ 15,  40,  52],
        [ 12,  42,  51],
        ...,
        [124, 198, 186],
        [131, 202, 190],
        [131, 202, 190]],

       ...,

       [[ 65, 100,  73],
        [ 65, 100,  73],
        [ 86,  99,  67],
        ...,
        [ 53,  96,  77],
        [ 59, 102,  79],
        [ 59, 102,  79]],

       [[ 73, 100,  73],
        [ 73, 100,  73],
        [116, 109,  70],
        ...,
        [ 54,  98,  75],
        [ 55,  99,  76],
        [ 55,  99,  76]],

       [[ 85, 101,  76],
        [ 85, 101,  76],
        [131, 115,  80],
        ...,
        [ 53,  97,  73],
        [ 55,  99,  75],
        [ 55,  99,  75]]